# Recurrent Neural Network
This file traines a recurrent neural network on the melspectogram features

In [1]:
import json
import yaml
from pathlib import Path
from lib_util import utils, plot


import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping 
from keras.losses import BinaryCrossentropy
from keras.metrics import BinaryAccuracy

from keras.layers import (
    Input,
    GlobalAvgPool1D,
    Dense,
    Bidirectional,
    GRU,
    Dropout,
)
from keras.models import Model, load_model
from keras.optimizers import Adam

from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import clip_ops

%load_ext autoreload
%autoreload 2
%matplotlib inline

2023-01-18 16:00:27.506159: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Define crossentropy and accuracy metric for the training routine

For the metric we use a binary accuracy, for the loss a binary cross-entropy loss

# Define the model

define the RNN model structure 

In [2]:
class UniformPredictor:
    def __init__(self, n_classes):
        self.n_classes = n_classes
    
    def train(self):
        pass

    def predict(self, X):
        i = np.random.randint(0, self.n_classes)
        return np.random.random((X.shape[0], self.n_classes))

class MajorityPredictor:
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.most_common_class = -1
    
    def train(self, y):
        self.most_common_class = np.argmax(np.sum(y, axis=0))

    def predict(self, X):
        preds = np.zeros((X.shape[0], self.n_classes))
        preds[:, self.most_common_class] = 1
        return preds

# Main routine

Load the configuration for this model

In [3]:
config = utils.get_config('rnn')

# Extract the values
model_name = config['model_name']
batch_size = config['batch_size']
epochs = config['epochs']

# Load preprocessed data

Load the numpy arrays and the label-class mapping. Split the data accordingly into train, validation and test set

In [4]:
melspec_data = utils.load_sliced_numpy_array('melspec_features')
labels = np.load('data/labels.npy')

labels_to_id = utils.get_class_mapping()

mel_train, mel_test_val, lab_train, lab_test_val = train_test_split(melspec_data, labels, train_size=config['train_set_size'], random_state=config['random_state'])
mel_val, mel_test, lab_val, lab_test             = train_test_split(mel_test_val, lab_test_val, test_size=(config['val_set_size']/(1-config['train_set_size'])), shuffle=False)

# Check the shapes of the splitted sets
assert mel_train.shape[0] == lab_train.shape[0] and mel_test.shape[0] == lab_test.shape[0] and mel_val.shape[0] == lab_val.shape[0]
assert mel_train.shape[1] == mel_test.shape[1] == mel_val.shape[1] and lab_train.shape[1] == lab_test.shape[1] == lab_val.shape[1]

Load slice from file melspec_features_001.npy
Load slice from file melspec_features_002.npy
Load slice from file melspec_features_003.npy
Load slice from file melspec_features_004.npy
Load slice from file melspec_features_005.npy
Load slice from file melspec_features_006.npy
Load slice from file melspec_features_007.npy
Load slice from file melspec_features_008.npy
Load slice from file melspec_features_009.npy
Load slice from file melspec_features_010.npy
Load slice from file melspec_features_011.npy
Load slice from file melspec_features_012.npy
Loaded 12 files as slices, resulting shape: (4068, 33152)


# Evaluate the naive predictor

In [7]:
model = UniformPredictor(len(labels_to_id))

# read in the config
pp_config = utils.get_config('preprocess')
n_mels = pp_config['melspectogram']['n_mels']

mel_test = mel_test.reshape(mel_test.shape[0], -1, n_mels)

predict = lambda model, data: np.argmax(model.predict(data), axis=-1)

mel_test = mel_test.reshape(mel_test.shape[0], -1, n_mels)

#Loading in the model
# model = utils.load_model(model_name)

# Training prediction
y_pred_train = predict(model, mel_train)
y_true_train = np.argmax(lab_train, axis= -1)
print(f"ACCURACY FOR TRAIN SET {accuracy_score(y_true_train, y_pred_train)*100:.4f} %")
print(f"MACRO F1 SCORE FOR TRAIN SET {f1_score(y_true_train, y_pred_train, average='macro')*100:.4f} %")
print(f"MICRO F1 SCORE FOR TRAIN SET {f1_score(y_true_train, y_pred_train, average='micro')*100:.4f} %")
print(f"WEIGHTED F1 SCORE FOR TRAIN SET {f1_score(y_true_train, y_pred_train, average='weighted')*100:.4f} %")
print()

# Validation prediction
y_pred_val = predict(model, mel_val)
y_true_val = np.argmax(lab_val, axis= -1)
print(f"ACCURACY FOR VAL SET {accuracy_score(y_true_val, y_pred_val)*100:.4f} %")
print(f"MACRO F1 SCORE FOR VAL SET {f1_score(y_true_val, y_pred_val, average='macro')*100:.4f} %")
print(f"MICRO F1 SCORE FOR VAL SET {f1_score(y_true_val, y_pred_val, average='micro')*100:.4f} %")
print(f"WEIGHTED F1 SCORE FOR TRAIN SET {f1_score(y_true_val, y_pred_val, average='weighted')*100:.4f} %")
print()

# Test prediction
y_pred_test = predict(model, mel_test)
y_true_test = np.argmax(lab_test, axis= -1)
print(f"ACCURACY FOR TEST SET {accuracy_score(y_true_test, y_pred_test)*100:.4f} %")
print(f"MACRO F1 SCORE FOR TEST SET {f1_score(y_true_test, y_pred_test, average='macro')*100:.4f} %")
print(f"MICRO F1 SCORE FOR TEST SET {f1_score(y_true_test, y_pred_test, average='micro')*100:.4f} %")
print(f"WEIGHTED F1 SCORE FOR TEST SET {f1_score(y_true_test, y_pred_test, average='weighted')*100:.4f} %")

ACCURACY FOR TRAIN SET 19.3888 %
MACRO F1 SCORE FOR TRAIN SET 19.3537 %
MICRO F1 SCORE FOR TRAIN SET 19.3888 %
WEIGHTED F1 SCORE FOR TRAIN SET 19.4442 %

ACCURACY FOR VAL SET 23.2787 %
MACRO F1 SCORE FOR VAL SET 22.9621 %
MICRO F1 SCORE FOR VAL SET 23.2787 %
WEIGHTED F1 SCORE FOR TRAIN SET 23.6115 %

ACCURACY FOR TEST SET 20.9493 %
MACRO F1 SCORE FOR TEST SET 20.8653 %
MICRO F1 SCORE FOR TEST SET 20.9493 %
WEIGHTED F1 SCORE FOR TEST SET 21.1428 %


In [9]:
model = MajorityPredictor(len(labels_to_id))

# read in the config
pp_config = utils.get_config('preprocess')
n_mels = pp_config['melspectogram']['n_mels']

mel_test = mel_test.reshape(mel_test.shape[0], -1, n_mels)

model.train(lab_train)

predict = lambda model, data: np.argmax(model.predict(data), axis=-1)

mel_test = mel_test.reshape(mel_test.shape[0], -1, n_mels)

#Loading in the model
# model = utils.load_model(model_name)

# Training prediction
y_pred_train = predict(model, mel_train)
y_true_train = np.argmax(lab_train, axis= -1)
print(f"ACCURACY FOR TRAIN SET {accuracy_score(y_true_train, y_pred_train)*100:.4f} %")
print(f"MACRO F1 SCORE FOR TRAIN SET {f1_score(y_true_train, y_pred_train, average='macro')*100:.4f} %")
print(f"MICRO F1 SCORE FOR TRAIN SET {f1_score(y_true_train, y_pred_train, average='micro')*100:.4f} %")
print(f"WEIGHTED F1 SCORE FOR TRAIN SET {f1_score(y_true_train, y_pred_train, average='weighted')*100:.4f} %")
print()

# Validation prediction
y_pred_val = predict(model, mel_val)
y_true_val = np.argmax(lab_val, axis= -1)
print(f"ACCURACY FOR VAL SET {accuracy_score(y_true_val, y_pred_val)*100:.4f} %")
print(f"MACRO F1 SCORE FOR VAL SET {f1_score(y_true_val, y_pred_val, average='macro')*100:.4f} %")
print(f"MICRO F1 SCORE FOR VAL SET {f1_score(y_true_val, y_pred_val, average='micro')*100:.4f} %")
print(f"WEIGHTED F1 SCORE FOR TRAIN SET {f1_score(y_true_val, y_pred_val, average='weighted')*100:.4f} %")
print()

# Test prediction
y_pred_test = predict(model, mel_test)
y_true_test = np.argmax(lab_test, axis= -1)
print(f"ACCURACY FOR TEST SET {accuracy_score(y_true_test, y_pred_test)*100:.4f} %")
print(f"MACRO F1 SCORE FOR TEST SET {f1_score(y_true_test, y_pred_test, average='macro')*100:.4f} %")
print(f"MICRO F1 SCORE FOR TEST SET {f1_score(y_true_test, y_pred_test, average='micro')*100:.4f} %")
print(f"WEIGHTED F1 SCORE FOR TEST SET {f1_score(y_true_test, y_pred_test, average='weighted')*100:.4f} %")

ACCURACY FOR TRAIN SET 24.2712 %
MACRO F1 SCORE FOR TRAIN SET 7.8123 %
MICRO F1 SCORE FOR TRAIN SET 24.2712 %
WEIGHTED F1 SCORE FOR TRAIN SET 9.4807 %

ACCURACY FOR VAL SET 21.6393 %
MACRO F1 SCORE FOR VAL SET 7.1159 %
MICRO F1 SCORE FOR VAL SET 21.6393 %
WEIGHTED F1 SCORE FOR TRAIN SET 7.6992 %

ACCURACY FOR TEST SET 26.0229 %
MACRO F1 SCORE FOR TEST SET 8.2597 %
MICRO F1 SCORE FOR TEST SET 26.0229 %
WEIGHTED F1 SCORE FOR TEST SET 10.7471 %
